# Using as instance properties

What happens when we use `__get__` and `__set__` and also constrain the instance dictionary with `__slots__`? Is not guaranteed to have the instance dictionary available. 


In [1]:
class IntegerValue:
    def __set__(self, instance, value):
        instance.stored_value = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        return getattr(instance, 'stored_value')

In [2]:
class Point1D:
    x = IntegerValue()

In [3]:
p1, p2 = Point1D(), Point1D()

p1.x = 10
p2.x = 20

In [4]:
p1.x, p2.x

(10, 20)

In [5]:
p1.__dict__, p2.__dict__

({'stored_value': 10}, {'stored_value': 20})

Appears to work just fine. But the descriptor is hardcode to be stored in a key in the instance dictinary

In [12]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [13]:
p = Point2D()

p.x = 10
p.y = 20 

In [14]:
p.__dict__

{'stored_value': 20}

In [15]:
p.x

20

That's the issue. If we hardcode the "stored_value" it can only be used a single time. Different instances tat uses the same descriptor in the same class will be wrongly overwritten. This way, we need differnt storage names for different properties

In [16]:
class IntegerValue:
    def __init__(self, name: str):
        self.storage_name = '_' + name 

    def __set__(self, instance, value):
        # it can't have the "storage_name hardcoded", will be leading to the same problem
        setattr(instance, self.storage_name, value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        return getattr(instance, self.storage_name, None)

In [18]:
class Point2D:
    x = IntegerValue('x')
    y = IntegerValue('y')

p1, p2 = Point2D(), Point2D()

p1.x = 10.1
p1.y = 20.1 

p2.x = 300
p2.y = 400

In [19]:
p1.x, p1.y

(10.1, 20.1)

In [20]:
p2.x, p2.y

(300, 400)

In [21]:
p1.__dict__, p2.__dict__

({'_x': 10.1, '_y': 20.1}, {'_x': 300, '_y': 400})

What happen if I use slots? it will not work, bc there's no way to add the '_x' into the instance dictionary, unless the variable name is defined in the slots

Insted of store in the instance, we need to create in the actual dictionary of the descriptor instace. HAS TO BE UNIQUE BY INSTANCE 

In [31]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        return self.values.get(instance, None)

In [37]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

p1, p2 = Point2D(), Point2D()

p1.x = 10 
p1.y = 20 

p2.x = 30 
p2.y = 40

In [38]:
p1.x, p2.x

(10, 30)

In [39]:
p1.y, p2.y

(20, 40)

In [40]:
p1.__dict__, p2.__dict__

({}, {})

In [41]:
Point2D.x.values

{<__main__.Point2D at 0x28b9244aba0>: 10,
 <__main__.Point2D at 0x28b92578e10>: 30}